In [34]:
from elasticsearch import Elasticsearch
import operator

es = Elasticsearch([{'host': 'lame14.enst.fr', 'port': 50014}])
index_from = 'info_trafic'
index_to = 'stif_info_trafic'

In [35]:
def read_info_trafic_page(size, from_, query):
    result = []
    res = es.search(index=index_from, body={"query": query}, size=size, from_=from_)
    res = res['hits']['hits']
    for i in range(0, len(res)):
        result = reduce(operator.add, [result, res[i]['_source']['infos']])
    return result

res = read_info_trafic_page(1000, 0, {"matchAll": {}})

In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}
es.indices.create(index=index_to, ignore=400, body=index_settings)

{u'acknowledged': True}

In [37]:
for item in res:
    res = es.index(index=index_to, doc_type='stif_info_trafic', body=item, id=item["pub_date"])